## `Pastikan klik 'File > Save a copy in Drive' sebelum memulai pengerjaan`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay

#Dataset

Data yang digunakan adalah data yang berasal dari Tokopedia (***bukan data sesungguhnya***). Mengenai penjelasan dataset adalah sebagai berikut:

|variable                       |class     |description |
|:------------------------------|:---------|:-----------|
**order_detail:**
id 			|object| angka unik dari order / id_order
customer_id 		|object|angka unik dari pelanggan
order_date 		|object| tanggal saat dilakukan transaksi
sku_id 			|object| angka unik dari produk (sku adalah stock keeping unit)
price			|int64| harga yang tertera pada tagging harga
qty_ordered 		|int64| jumlah barang yang dibeli oleh pelanggan
before_discount	|float64| nilai harga total dari produk (price * qty_ordered)
discount_amount	|float64| nilai diskon product total
after_discount		|float64| nilai harga total produk ketika sudah dikurangi dengan diskon
is_gross 		|int64| menunjukkan pelanggan belum membayar pesanan
is_valid		|int64| menunjukkan pelanggan sudah melakukan pembayaran
is_net			|int64| menunjukkan transaksi sudah selesai
payment_id 		|int64| angka unik dari metode pembayaran
||
**sku_detail:**
id |object| angka unik dari produk (dapat digunakan untuk key saat join)
sku_name 		|object| nama dari produk
base_price		|float64| harga barang yang tertera pada tagging harga / price
cogs 			|int64| cost of goods sold / total biaya untuk menjual 1 produk
category		|object| kategori produk
||
**customer_detail:**
id 			|object| angka unik dari pelanggan
registered_date	|object| tanggal pelanggan mulai mendaftarkan diri sebagai anggota
||
**payment_detail:**
id			|int64| angka unik dari metode pembayaran
payment_method	|object| metode pembayaran yang digunakan


In [2]:
#Sumber data yang digunakan
path_od = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/order_detail.csv"
path_pd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/payment_detail.csv"
path_cd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/customer_detail.csv"
path_sd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/sku_detail.csv"
df_od = pd.read_csv(path_od)
df_pd = pd.read_csv(path_pd)
df_cd = pd.read_csv(path_cd)
df_sd = pd.read_csv(path_sd)

In [22]:
#Mengampilkan 5 baris pertama
df_od.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4


In [23]:
#Mengampilkan 5 baris pertama
df_pd.head()

,id,payment_method
0,1,cod
1,2,jazzvoucher
2,3,customercredit
3,4,Payaxis
4,5,jazzwallet


In [24]:
#Mengampilkan 5 baris pertama
df_cd.head()

,id,registered_date
0,C996508L,2021-07-10
1,C180415L,2021-07-18
2,C535451L,2021-07-23
3,C177843L,2021-07-12
4,C951682L,2021-07-27


In [25]:
#Mengampilkan 5 baris pertama
df_sd.head()

,id,sku_name,base_price,cogs,category
0,P798444,AT-FSM-35,57631.70,46052,Kids & Baby
1,P938347,AYS_Haier-18HNF,3931789.26,3499256,Appliances
2,P826364,Atalian _DV206A-Brown-41,324597.00,243426,Men Fashion
3,P467533,Darul_Sakoon_Food_Bundle,2870.42,2378,Superstore
4,P229955,HP_15AY-15-Ay072NIA-ci3,2265625.00,1631250,Computing


In [3]:
#Menjalankan SQL di Colab
from sqlite3 import connect
conn = connect(':memory:')
df_od.to_sql('order_detail',conn, index=False, if_exists='replace')
df_pd.to_sql('payment_detail', conn, index=False, if_exists='replace')
df_sd.to_sql('sku_detail', conn, index=False, if_exists='replace')
df_cd.to_sql('customer_detail', conn, index=False, if_exists='replace')

3998

In [4]:
#Query SQL untuk menggabungkan data
df = pd.read_sql("""
SELECT
    order_detail.*,
    payment_detail.payment_method,
    sku_detail.sku_name,
    sku_detail.base_price,
    sku_detail.cogs,
    sku_detail.category,
    customer_detail.registered_date
FROM order_detail
LEFT JOIN payment_detail
    on payment_detail.id = order_detail.payment_id
LEFT JOIN sku_detail
    on sku_detail.id = order_detail.sku_id
LEFT JOIN customer_detail
    on customer_detail.id = order_detail.customer_id
""", conn)

In [28]:
#Mengampilkan 5 baris pertama
df.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id,payment_method,sku_name,base_price,cogs,category,registered_date
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5,jazzwallet,RB_Dettol Germ Busting Kit-bf,26100.0,18270,Others,2021-07-07
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5,jazzwallet,PS4_Slim-500GB,1971942.0,1321182,Entertainment,2021-11-20
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4,Payaxis,Changhong Ruba 55 Inches UD55D6000i Ultra HD T...,7482000.0,5162580,Entertainment,2021-11-19
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5,jazzwallet,dawlance_Inverter 30,3593680.0,3054628,Appliances,2021-11-03
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4,Payaxis,Dawlance_Inverter-45 2.0 ton,4413220.0,3177472,Appliances,2021-07-05


In [11]:
#Menampikan tipe data tiap kolom
df.dtypes

id                            str
customer_id                   str
order_date         datetime64[us]
sku_id                        str
price                       int64
qty_ordered                 int64
before_discount             int64
discount_amount             int64
after_discount              int64
is_gross                    int64
is_valid                    int64
is_net                      int64
payment_id                  int64
payment_method                str
sku_name                      str
base_price                  int64
cogs                        int64
category                      str
registered_date    datetime64[us]
dtype: object

In [10]:
#Mengubah tipe data agar mudah dilakukan pengolahan data
df = df.astype(
    {"before_discount":'int', "discount_amount":'int', 
     "after_discount":'int',"base_price":'int'})
df.dtypes

id                            str
customer_id                   str
order_date         datetime64[us]
sku_id                        str
price                       int64
qty_ordered                 int64
before_discount             int64
discount_amount             int64
after_discount              int64
is_gross                    int64
is_valid                    int64
is_net                      int64
payment_id                  int64
payment_method                str
sku_name                      str
base_price                  int64
cogs                        int64
category                      str
registered_date    datetime64[us]
dtype: object

In [9]:
#Mengubah tipe kolom Date menjadi Datetime
df['order_date']= pd.to_datetime(df['order_date'])
df['registered_date']= pd.to_datetime(df['registered_date'])
df.dtypes

id                            str
customer_id                   str
order_date         datetime64[us]
sku_id                        str
price                       int64
qty_ordered                 int64
before_discount           float64
discount_amount           float64
after_discount            float64
is_gross                    int64
is_valid                    int64
is_net                      int64
payment_id                  int64
payment_method                str
sku_name                      str
base_price                float64
cogs                        int64
category                      str
registered_date    datetime64[us]
dtype: object

In [8]:
#Mengecek duplicate
df.duplicated(keep=False).sum()

np.int64(0)

In [7]:
#Mengecek Null
df.isna().sum()

id                 0
customer_id        0
order_date         0
sku_id             0
price              0
qty_ordered        0
before_discount    0
discount_amount    0
after_discount     0
is_gross           0
is_valid           0
is_net             0
payment_id         0
payment_method     0
sku_name           0
base_price         0
cogs               0
category           0
registered_date    0
dtype: int64

## `No 1`
**Dear Data Analyst**,
<br></br>
Akhir tahun ini, perusahaan akan memberikan hadiah bagi pelanggan yang memenangkan kompetisi **Festival Akhir Tahun**. Tim Marketing membutuhkan bantuan untuk menentukan perkiraan hadiah yang akan diberikan pada pemenang kompetisi nantinya. Hadiah tersebut akan diambil dari **TOP 5 Produk** dari Kategori **Mobiles & Tablets** selama tahun 2022, dengan jumlah kuantitas penjualan (valid = 1) paling tinggi.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data dengan Category = Mobiles & Tablets
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name
5. Gunakan sort_values untuk mengurutkan data
6. Gunakan head untuk menampilkan top 5 produk


```



</details>

## `Jawaban No 1`

In [13]:
df_filter = df[
 (df["is_valid"] == 1) &
 (df["category"] == "Mobiles & Tablets") &
 (df["order_date"].dt.year == 2022)
]

df_top_5_produk = (
    df_filter
    .groupby(by=["sku_name"], as_index=False)["qty_ordered"].sum()
    .sort_values(by="qty_ordered", ascending=False)
    .head(5)
)

df_top_5_produk

,sku_name,qty_ordered
1,IDROID_BALRX7-Gold,1000
2,IDROID_BALRX7-Jet black,31
3,Infinix Hot 4-Gold,15
43,samsung_Grand Prime Plus-Black,11
34,infinix_Zero 4-Grey,10


## `No 2`
**Dear Data Analyst**,
<br></br>
Menindaklanjuti meeting gabungan Tim Werehouse dan Tim Marketing, kami menemukan bahwa ketersediaan stock produk dengan Kategori Others pada akhir 2022 kemarin masih banyak.
1. Kami mohon bantuan untuk melakukan pengecekan data penjualan kategori tersebut dengan tahun 2021 secara kuantitas penjualan. Dugaan sementara kami, telah terjadi penurunan kuantitas penjualan pada 2022 dibandingkan 2021. (Mohon juga menampilkan data ke-15 kategori)
<br></br>
2. Apabila memang terjadi penurunan kuantitas penjualan pada kategori Others, kami mohon bantuan untuk menyediakan data TOP 20 nama produk yang mengalami penurunan paling tinggi pada 2022 jika dibanding dengan 2021. Hal ini kami gunakan sebagai bahan diskusi pada meeting selanjutnya.  

Mohon bantuan untuk mengirimkan data tersebut paling lambat 4 hari dari hari ini. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Werehouse**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data pada transaksi selama 2021
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name masing-masing tahun
5. Menggabungkan kedua data dengan merge
6. Lakukan pengurangan kolom qty 2022 dengan qty 2021


```



</details>

## `Jawaban No 2.1`

In [14]:
df_filter_2021 = df[
 (df["is_valid"] == 1) &
 (df["order_date"].dt.year == 2021)
] 
df_filter_2022 = df[
 (df["is_valid"] == 1) & 
 (df["order_date"].dt.year == 2022)
]
qty_2021 = df_filter_2021.groupby(by=["category"], as_index=False)["qty_ordered"].sum()
qty_2022 = df_filter_2022.groupby(by=["category"], as_index=False)["qty_ordered"].sum()

merged_df = pd.merge(
    qty_2021.rename(columns={"qty_ordered": "qty_2021"}),
    qty_2022.rename(columns={"qty_ordered": "qty_2022"}),
    on="category",
    how="outer")
merged_df["selisih_qty"] = merged_df["qty_2022"] - merged_df["qty_2021"]
merged_df["pertumbuhan_qty"] = ((merged_df["qty_2022"] - merged_df["qty_2021"]) 
                                / merged_df["qty_2021"] * 100).round(2)
merged_df.sort_values(by="selisih_qty", ascending=True)

,category,qty_2021,qty_2022,selisih_qty,pertumbuhan_qty
10,Others,426,263,-163,-38.26
12,Soghaat,759,612,-147,-19.37
8,Men Fashion,237,175,-62,-26.16
1,Beauty & Grooming,168,153,-15,-8.93
2,Books,171,195,24,14.04
0,Appliances,124,148,24,19.35
5,Health & Sports,173,200,27,15.61
3,Computing,109,153,44,40.37
11,School & Education,184,237,53,28.80
6,Home & Living,193,250,57,29.53


## `Jawaban No 2.2`

In [15]:
df_others = df[
    (df['is_valid'] == 1) &
    (df['category'] == 'Others') &  
    (df['order_date'].dt.year.isin([2021, 2022]))
]
df_others['year'] = df_others['order_date'].dt.year

others_product = (
    df_others
    .groupby(['sku_name', 'year'])["qty_ordered"].sum()
    .reset_index()
    .pivot(index='sku_name', columns='year', values='qty_ordered')
    .fillna(0)
    .reset_index()
    .rename(columns={2021: 'qty_ordered_21', 2022: 'qty_ordered_22'})
)
others_product.columns.name = None

others_product['selisih_qty_others'] = (
    others_product['qty_ordered_22'] - others_product['qty_ordered_21']
)

result = others_product.sort_values(by='selisih_qty_others', ascending=True).head(20)
result

,sku_name,qty_ordered_21,qty_ordered_22,selisih_qty_others
69,RB_Dettol Germ Busting Kit-bf,200.0,45.0,-155.0
18,Dawlance_MD 10 + DWB 600,23.0,0.0,-23.0
84,Telemall_MM-DR-HB-L,23.0,2.0,-21.0
137,iu_Tickets General Enclosure-Islamabad,20.0,0.0,-20.0
70,RS_Rehmat-e-Shereen Mix Mithai,13.0,0.0,-13.0
141,kansai_NeverWet,10.0,1.0,-9.0
154,sindbad_Sindbad Gift Card-3,7.0,0.0,-7.0
133,emart_00-1,7.0,1.0,-6.0
93,Vouch 365 2016,5.0,0.0,-5.0
4,Am-PTV_ATS-004-M,5.0,0.0,-5.0


## `No 3`
**Dear Data Analyst**,
<br></br>
Terkait ulang tahun perusahaan pada 2 bulan mendatang, Tim Digital Marketing akan memberikan informasi promo bagi pelanggan pada akhir bulan ini. Kriteria pelanggan yang akan kami butuhkan adalah mereka yang sudah melakukan check-out namun belum melakukan pembayaran (is_gross = 1) selama tahun 2022. Data yang kami butuhkan adalah ID Customer dan Registered Date.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Digital Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Digital Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan gross = 1
2. Memfilter data dengan valid = 0
3. Memfilter data dengan net = 0
4. Memfilter data pada transaksi selama 2022


```



</details>

## `Jawaban No 3`

In [16]:
df_filter = df[
 (df['is_gross'] == 1) &
 (df['is_valid'] == 0) &
 (df['is_net'] == 0) &
 (df['order_date'].dt.year == 2022)
]

pelanggan_checkout = (
    df_filter[['customer_id', 'registered_date']]
    .sort_values(by='registered_date')
    .drop_duplicates()
    .reset_index(drop=True)
)

pelanggan_checkout

,customer_id,registered_date
0,C349980L,2021-07-02
1,C849967L,2021-07-02
2,C378684L,2021-07-03
3,C831683L,2021-07-03
4,C893754L,2021-07-03
...,...,...
815,C877563L,2022-08-24
816,C788348L,2022-08-25
817,C741725L,2022-08-25
818,C925840L,2022-08-28


In [ ]:
#Jalankan kode ini untuk mendownload file
from google.colab import files
[nama variabel file].to_csv('audience_list.csv', encoding = 'utf-8-sig',index=False) #ganti [nama variabel file] dengan nama variabel yang digunakan
files.download('audience_list.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## `No 4`
**Dear Data Analyst**,
<br></br>
Pada bulan October hingga Desember 2022, kami melakukan campaign setiap hari Sabtu dan Minggu. Kami hendak menilai, apakah campaign tersebut cukup berdampak pada kenaikan penjualan (before_discount). Mohon bantuan untuk menampilkan data:
1. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) per bulan tersebut. Apakah ada peningkatan penjualan pada masing-masing bulan tersebut.
2. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) keseluruhan 3 bulan tersebut.

Mohon bantuan untuk mengirimkan data tersebut paling lambat minggu depan. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Campaign**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Buatlah kolom baru untuk day, month, dan month number
2. Memfilter data dengan valid = 1
3. Memfilter data dengan day termasuk 'Saturday', 'Sunday' dengan menggunakan fungsi isin dan simpan dalam variable 'weekends'
4. Memfilter data dengan day termasuk 'Monday','Tuesday','Wednesday','Thusday','Friday' dengan menggunakan fungsi isin dan simpan dengan variabel 'weekday'
5. Memfilter data pada transaksi selama 2022 bulan Oktober sampai dengan Desember
6. Gunakan groupby berdasarkan pada masing-masing variabel
7. Menggabungkan kedua data dengan merge
8. Tampilkan pada grafik batang untuk melihat perbedaananya


```



</details>

## `Jawaban No 4.1`

In [18]:
df['day'] =  df['order_date'].dt.day_name()
df['month'] =  df['order_date'].dt.month_name()
df['month_number'] = df['order_date'].dt.month
df['year'] = df['order_date'].dt.year

df_valid = df[df['is_valid'] == 1].copy()

weekend = df_valid[df_valid['day'].isin(['Saturday', 'Sunday'])]
weekday = df_valid[df_valid['day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]

month_filter = [10,11,12]
df_filter = df_valid[
    (df_valid['year'] == 2022) &
    (df_valid['month_number'].isin(month_filter))]

weekend_filter = weekend[
    (weekend['year'] == 2022) &
    (weekend['month_number'].isin(month_filter))]
weekday_filter = weekday[
    (weekday['year'] == 2022) &
    (weekday['month_number'].isin(month_filter))]

weekend_monthly = (weekend_filter
    .groupby(['month_number', 'month'])
    .agg(
        total_weekend_sales=('before_discount', 'sum'), 
        total_days=('order_date', 'nunique'))
    .reset_index().sort_values('month_number'))
weekend_monthly['avg_weekend_sales'] = weekend_monthly['total_weekend_sales'] / weekend_monthly['total_days']

weekday_monthly = (weekday_filter
    .groupby(['month_number', 'month'])
    .agg(
        total_weekday_sales=('before_discount', 'sum'), 
        total_days=('order_date', 'nunique'))
    .reset_index().sort_values('month_number'))
weekday_monthly['avg_weekday_sales'] = weekday_monthly['total_weekday_sales'] / weekday_monthly['total_days']


merged_monthly = pd.merge(
    weekend_monthly,
    weekday_monthly,
    on=['month', 'month_number'], 
    how='outer')
merged_monthly['increase'] = (
    merged_monthly['avg_weekend_sales']-merged_monthly['avg_weekday_sales']) 

result = merged_monthly[['month_number', 'month', 'avg_weekend_sales', 'avg_weekday_sales', 'increase']].copy()
result.sort_values('month_number', ascending=True)


,month_number,month,avg_weekend_sales,avg_weekday_sales,increase
2,10,October,5.708341e+06,7872212.4,-2.163872e+06
1,11,November,5.774045e+06,6204666.0,-4.306210e+05
0,12,December,4.105994e+06,8542530.0,-4.436536e+06


## `Jawaban No 4.2`

In [19]:
df['day'] =  df['order_date'].dt.day_name()
df['month'] =  df['order_date'].dt.month_name()
df['month_number'] = df['order_date'].dt.month
df['year'] = df['order_date'].dt.year

df_valid = df[df['is_valid'] == 1].copy()

weekend = df_valid[df_valid['day'].isin(['Saturday', 'Sunday'])]
weekday = df_valid[df_valid['day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]

month_filter = [10,11,12]
df_filter = df_valid[
    (df_valid['year'] == 2022) &
    (df_valid['month_number'].isin(month_filter))]

weekend_filter = weekend[
    (weekend['year'] == 2022) &
    (weekend['month_number'].isin(month_filter))]
weekday_filter = weekday[
    (weekday['year'] == 2022) &
    (weekday['month_number'].isin(month_filter))]

weekend_monthly = (weekend_filter
    .groupby(['month_number', 'month'])
    .agg(
        total_weekend_sales=('before_discount', 'sum'), 
        total_days=('order_date', 'nunique'))
    .reset_index().sort_values('month_number'))
weekend_monthly['avg_weekend_sales'] = weekend_monthly['total_weekend_sales'] / weekend_monthly['total_days']

weekday_monthly = (weekday_filter
    .groupby(['month_number', 'month'])
    .agg(
        total_weekday_sales=('before_discount', 'sum'), 
        total_days=('order_date', 'nunique'))
    .reset_index().sort_values('month_number'))
weekday_monthly['avg_weekday_sales'] = weekday_monthly['total_weekday_sales'] / weekday_monthly['total_days']

total_weekend_sales = weekend_filter['before_discount'].sum()
total_weekend_days = weekend_filter['order_date'].nunique()
avg_weekend_overall = total_weekend_sales / total_weekend_days 

total_weekday_sales = weekday_filter['before_discount'].sum()
total_weekday_days = weekday_filter['order_date'].nunique()
avg_weekday_overall = total_weekday_sales / total_weekday_days 


overall_increase = (
    (avg_weekend_overall - avg_weekday_overall) / avg_weekday_overall 
    * 100)

overall_summary = pd.DataFrame({
    'period': ['Q4_2022'],
    'avg_weekends': [round(avg_weekend_overall, 2)],
    'avg_weekdays': [round(avg_weekday_overall, 2)],
    'increase_%': [round(overall_increase, 2)]
})

overall_summary


,period,avg_weekends,avg_weekdays,increase_%
0,Q4_2022,5269300.0,7520249.29,-29.93
